Todo:
- pohlídat oscrapování odkazů nahoře (řečník: Minulý) / vyřešeno, ale teď se ztrácí "(pokračuje Tomio Okamura)" zde: (pokračuje Tomio Okamura) – protože to není v odstavci

- obrácené vnoření A a B v 1998ps_stenprot_003schuz_s003023.htm
- "(pokračuje" v 1920ns_se_182schuz_s182005
- procedurální věci v 1945pns_stenprot_003schuz_s003001.tm
-  Federální shromáždění ČSFR, 1992 – 1992/ 
Sněmovna liud nemá ani tučné, ani odkazů

In [79]:
import os
import csv
import json
import random
from bs4 import BeautifulSoup
import concurrent.futures

In [80]:
kam = "data_raw/schuze"

try:
    os.makedirs(kam)
except:
    pass

In [81]:
import re

In [82]:
# re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", "zxz").group()

In [83]:
mesice = {
    'ledna':1,
    'února':2,
    'března':3,
    'dubna':4,
    'května':5,
    'června':6,
    'července':7,
    'srpna':8,
    'září':9,
    'října':10,
    'listopadu':11,
    'prosince':12
}

funkce = [
    'předseda',
    'předsedkyně',
    'předs.',
    'místopředseda',
    'místopředsedkyně',
    'podpředseda',
    'podpředsedkyně'
    'poslanec',
    'poslankyně',
    'posl.',
    'ministr',
    'ministryně',
    'zpravodaj',
    'zpravodajka',
    'zástupce',
    'zástupkyně',
    'sen.',
    'senátor',
    'senátorka',
    'společná',
    'společná',
    'tajemník',
    'tajemnice',
    'ministerský',
    'ministerská',
    'guvernér'
]

In [84]:
def oscrapovani_stenozaznamu(soubor, zdroj=False):
    
    def ziskej_metadata(soubor):
        if "psse" not in soubor:
            komora_kratce = soubor.split("_")[0][4:]
        else:
            komora_kratce = "psse"
        obdobi = soubor.split("_")[0][0:4]
        schuze = soubor.split("_")[-1][1:4]
        cast = soubor.split("_")[-1][-7:-4]

        return komora_kratce, int(obdobi), int(schuze), int(cast.removesuffix("."))

    def najdi_datum(polivka):
        

        surove_datum = ""
        if polivka.find(class_="date"):
            surove_datum = polivka.find(class_="date").text.replace('\xa0',' ').lower()
        if surove_datum == "":
            try:
                surove_datum = re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", polivka.text).group()
            except:
                surove_datum = polivka.find("a", href="../index.htm").text.lower()

        try:
            rok = int(surove_datum.split(" ")[-1])
            mesic = surove_datum.split(" ")[-2]        
            den = int(surove_datum.split(" ")[-3].removesuffix('.'))
            return f"{rok}-{mesice[mesic]:02}-{den:02}"
        except:
            return None

    def najdi_komoru_a_schuzi(polivka):
        odkazy = polivka.find_all('a')
        komora, schuze = None, None
        try:
            komora = soup.find("h1").text.strip()
        except:
            komora = soup.find(id="breadcrumb").find_all("a")[1].text.strip()
        finally:
            pass
        for a in odkazy:
            if 'href' in a.attrs:
                if './index.htm' in a['href']:
                    schuze = a.text.strip()
        return komora, schuze

    def ciste_odstavce(seznam):
        return ' '.join(seznam).replace('\xa0',' ').replace('***','').removeprefix(':').strip()
    
    metadata = ziskej_metadata(soubor)

    slovnik_metadata = {
        'komora': metadata[0],
        'obdobi': metadata[1],
        'schuze': metadata[2],
        'cast': metadata[3],
        'soubor': soubor
    }

    if "1996ps" in soubor:
        parsovatko = "html5lib" # kvůli 1996ps_stenprot_016schuz_s016005.htm aj., odkazy začínají mimo odstavce
    else:
        parsovatko = "lxml"
    
    with open(os.path.join("downloads/schuze/",soubor), "r", encoding="windows-1250") as schuze:
        schuze = schuze.read().replace('\n',' ')
        soup = BeautifulSoup(schuze, parsovatko)

    slovnik_metadata['datum'] = najdi_datum(soup)
    slovnik_metadata['komora_komplet'], slovnik_metadata['schuze_komplet'] = najdi_komoru_a_schuzi(soup)
    slovnik_metadata['prepsano'] = True
    slovnik_metadata['autorizovano'] = True

    orez = 130
    
    projevy = []
    
    if "nebyl dosud přepsán. Prosím, zkuste později." in soup.text:
        slovnik_metadata['prepsano'] = False
        return slovnik_metadata, projevy
        
    if ("Neprošlo jazykovou korekturou, neautorizováno" in soup.text) or ("Neprošlo opravou po digitalizaci" in soup.text):
        slovnik_metadata['autorizovano'] = False
    
    uz_byl_tucnej = False
    poradi = 1
    projev = {'mluvci':None,'poradi':poradi,'soubor':soubor,'mluvci_id':None}
    odstavce = []

    vsechny_odstavce = soup.find_all("p")

    zacatek_dokumentu = 5
    for index, p in enumerate(vsechny_odstavce[0:7]):
        if ("Minulý" in p.text) and ("Dolů" in p.text) and ("Další" in p.text) and (len(p.text.strip()) < 60):
            zacatek_dokumentu = index + 1
    
    for p in vsechny_odstavce[zacatek_dokumentu:-10]:
        if zdroj == True:
            print("____" + p.text[0:orez])

        if ("1992fs_sl" in soubor) or ("1990fs_sl" in soubor):

            if (':' in p.text.strip()[0:orez - 30]) and (len([l for l in p.text.strip().split(":")[0] if l.isupper()]) > 1):
                
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                uz_byl_tucnej = True
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.text.split(':')[0].strip()
                projev['mluvci_id'] = None
                odstavce.append(' '.join(p.text.split(':')[1:]).strip())

            else:
                odstavce.append(p.text.strip())

        else:

            if zdroj:
                if p.find("b"):
                    print("TUČNÉ")

            if ':' in p.text.strip()[0:orez] and (
                    p.find('b', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False) 
                    or 
                    p.find('u', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False)
                ) and (
                    "výsledkem:" not in p.text # kvůli 2006se_stenprot_015schuz_s015018.htm
                ):

                pocitadlo_zvyrazneni = 0
                prvni_tucne_je_druhe_tucne = False
                zadna_zminka_o_funkci = True
                
                if len(p.find_all("b")) > 1: # kvůli 1920ns_se_stenprot_195schuz_s195001.htm
                    if zdroj == True:
                        print("počítám tučnost")
                    for tucne in p.find_all("b"):
                        if tucne.text.strip() in p.text.strip()[0:orez]:
                            pocitadlo_zvyrazneni += 1
                    if p.find_all("b")[0].text.strip() == p.find_all("b")[1].text.strip(): # kvůli 1996ps_stenprot_021schuz_s021021.htm
                        prvni_tucne_je_druhe_tucne = True
                    for f in funkce:
                        if f == p.text.strip().split(' ')[0].strip().lower(): # kvůli 1948ns_stenprot_054schuz_s054003.htm: Podpředseda Valo: Ďalším prihláseným rečníkom je pán posl. Navračič. Prosím, aby sa ujal slova.
                            if zdroj == True:
                                print(f.upper())
                            zadna_zminka_o_funkci = False
                if (pocitadlo_zvyrazneni > 1) & (prvni_tucne_je_druhe_tucne == False) & (zadna_zminka_o_funkci == True):
                    if zdroj == True:
                        print("dvakrát, bez akce")
                    odstavce.append(p.text.strip())

                else:
                    if zdroj == True:
                        print('mluvčí (: + a/b)')

                    if ("resoluční návrh" in p.text.lower()) or ("pozměňovací návrh" in p.text.lower()) or ("doplňovací návrh" in p.text.lower()):
                        odstavce.append(p.text.strip())
                        # kvůli 1920ns_ps_stenprot_078schuz_s078006.htm

                    else:
                        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.text.split(':')[0].strip()
                        try:
                            projev['mluvci_id'] = odstavec.find("a")['href'].split('?')[-1]
                        except:
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

            elif p.find("b") and (len(p.text.strip()) < 100) and (p.text.strip().endswith(':')):
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.find('b').text.replace(':','').strip()
    
            elif p.find('a'): ## častá chyba, viz testy: je odkaz, není dvojtečka. kód se tu zbytečně duplikuje, whatever
                if zdroj == True:
                    print('ODKAZ')
                if p.text.strip().startswith(p.find('a').text.strip()) and (p.find("a").get("id") != "_d") and(p.find("a").get("name") != "OLE_LINK1"):
                    # p.find("a").get("name") != "OLE_LINK1" kvůli 1946uns_stenprot_087schuz_s087005.htm
                    
                    if zdroj == True:
                        print('mluvčí p.find(a)')
    
                    projev['text'] = ciste_odstavce(odstavce)
                    if projev['text'] != '':
                        poradi += 1
                        projevy.append(projev)
                    projev = {}
                    odstavce = []
                    projev['poradi'] = poradi
                    projev['soubor'] = soubor
                    projev['mluvci'] = p.find("a").text.strip()
                    try:
                        projev['mluvci_id'] = odstavec.find("a")['href'].split('?')[-1]
                    except:
                        projev['mluvci_id'] = None
                    odstavce.append(p.text.replace(projev['mluvci'],'').strip())

                else:
                    odstavce.append(p.text.strip())
                    
            else:
                odstavce.append(p.text.strip())
        
    projev['text'] = ciste_odstavce(odstavce)
    projevy.append(projev)    
    
    return slovnik_metadata, projevy

In [85]:
def html2data(soubor):
    
    ciste = oscrapovani_stenozaznamu(soubor)
    soubor = soubor.replace('.htm','')
    
    with open(os.path.join(kam, f'prepis_{soubor}.ndjson'), "w+", encoding="utf-8") as export:
        for projev in ciste[1]:
            export.write(json.dumps(projev) + '\n')

    with open(os.path.join(kam, f'meta_{soubor}.csv'), "w+", newline="", encoding="utf-8") as export_file:
        fieldnames = ciste[0].keys()
        writer = csv.DictWriter(export_file, fieldnames=fieldnames, quoting=csv.QUOTE_STRINGS)
        writer.writeheader()
        writer.writerow(ciste[0])

In [86]:
oscrapovani_stenozaznamu("1948ns_stenprot_054schuz_s054003.htm", zdroj=True)

____
____ Ráda bych se zmínila ještě několika slovy o účasti žen při přechodu vesnice k socialismu. Uvedu příklad JZD Podůlšany, okres Pard
TUČNÉ
____ Loňského roku pracovalo v pardubickém kraji v 564 pracovních skupinách 4.875 žen. V letošním roce se tento počet daleko zvýší. Na
____ Našemu venkovu se otevřely široké prespektivy rozvinutí všech tvůrčích sil při budování nového života na naší vesnici. Osou tohot
____ Mírový rozpočet, který projednáváme a pro který budeme rádi hlasovat, je odrazem mírového rozpočtu, který byl schválen národy Sov
____ Podpředseda Valo: Ďalším prihláseným rečníkom je pán posl. Navračič. Prosím, aby sa ujal slova.  
TUČNÉ
počítám tučnost
PODPŘEDSEDA
mluvčí (: + a/b)
____ Posl. Navračič: Vážená snemovňa, panie a páni! 
TUČNÉ
mluvčí (: + a/b)
____ Dovoľte mi, aby som k rozpočtu na r. 1951 povedal niekoľko slov k rozpočtovej položke štátnych majetkov a štátnych strojových sta
____ Vezmime len ornú pôdu, ktorú teraz štátne majetky na Slovensku obhospodarujú, a

({'komora': 'ns',
  'obdobi': 1948,
  'schuze': 54,
  'cast': 3,
  'soubor': '1948ns_stenprot_054schuz_s054003.htm',
  'datum': '1951-03-21',
  'komora_komplet': 'NS RČS 1948-1954',
  'schuze_komplet': '54. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1948ns_stenprot_054schuz_s054003.htm',
   'mluvci_id': None,
   'text': 'Ráda bych se zmínila ještě několika slovy o účasti žen při přechodu vesnice k socialismu. Uvedu příklad JZD Podůlšany, okres Pardubice, kde družstevníci zásluhou žen pochopili, kdo jim zdržuje a rozbíjí JZD v jeho rozvoji. Zde dřívější statkář Vinař ještě s několika vesnickými boháči ovlivňovali JZD a byli brzdou při jeho přechodu na vyšší typ. Mohli tuto zákeřnou činnost provádět proto, že se muži nedovedli postavit v JZD se vší rozhodností proti jejich záškodnické práci. Již před zasedáním ÚV KSČ ženy v Podůlšanech, které mají v JZD většinu, prosazovaly svůj názor, že je třeba vyloučit vesnické boháče z druž

In [87]:
oscrapovani_stenozaznamu("1929ns_ps_stenprot_247schuz_s247002.htm", zdroj=True)

____
____ Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, c
____ Pokud jde o pojištění prostřednictvím státní záruky, přednesl bych zvláštní přání. Vývojem v jednotlivých státech, neschopností p
____ Dnešní své vývody končím přáním, aby náš těžce ohrožený vývozní průmysl získal oné podpory státu, která odpovídá správnému porozu
____ Místopředseda Špatný (zvoní): Dalším řečníkem je pan posl. Richter. Uděluji mu slovo.  
TUČNÉ
počítám tučnost
MÍSTOPŘEDSEDA
mluvčí (: + a/b)
____ Posl. Richter: Slavná sněmovno! Dvouparagrafový zákon, kterým se pro příští léta stanoví úhrn státních záruk za exportní úvěry, j
TUČNÉ
počítám tučnost
POSL.
mluvčí (: + a/b)
____ Ujal jsem se však slova z jiného důvodu, než abych rozebíral předloženou osnovu. Její přednosti i nedostatky už vytkli páni zprav
TUČNÉ
____ Považuji za jednu z nejdůležitějsích otázek pro naši hospodářskou politiku a naše národní hospodářství vyřešiti slušně

({'komora': 'ns',
  'obdobi': 1929,
  'schuze': 247,
  'cast': 2,
  'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
  'datum': '1933-02-10',
  'komora_komplet': 'NS RČS 1929-1935',
  'schuze_komplet': '247. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
   'mluvci_id': None,
   'text': 'Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, co jsem již řekl, že třeba velmi litovati, že tento úvěr přichází teprve nyní. Pokud jde o prvou část, o pojištění vývozních úvěrů ve vlastním slova smyslu, která se netýká zaručení úvěrů zahraničním veřejným korporacím, nýbrž soukromníkům, vlastně se zde o ni nejedná. Bylo by třeba, aby i v tomto oboru zavládla jakási velkorysost. Ovšem jest to velmi choulostivá otázka. Neboť právě tehdy, když jde o státní nebo veřejné peníze, mají lidé sklon k jakési rozhazovačnosti a velkor

In [88]:
oscrapovani_stenozaznamu("1929ns_ps_stenprot_247schuz_s247002.htm", zdroj=True)

____
____ Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, c
____ Pokud jde o pojištění prostřednictvím státní záruky, přednesl bych zvláštní přání. Vývojem v jednotlivých státech, neschopností p
____ Dnešní své vývody končím přáním, aby náš těžce ohrožený vývozní průmysl získal oné podpory státu, která odpovídá správnému porozu
____ Místopředseda Špatný (zvoní): Dalším řečníkem je pan posl. Richter. Uděluji mu slovo.  
TUČNÉ
počítám tučnost
MÍSTOPŘEDSEDA
mluvčí (: + a/b)
____ Posl. Richter: Slavná sněmovno! Dvouparagrafový zákon, kterým se pro příští léta stanoví úhrn státních záruk za exportní úvěry, j
TUČNÉ
počítám tučnost
POSL.
mluvčí (: + a/b)
____ Ujal jsem se však slova z jiného důvodu, než abych rozebíral předloženou osnovu. Její přednosti i nedostatky už vytkli páni zprav
TUČNÉ
____ Považuji za jednu z nejdůležitějsích otázek pro naši hospodářskou politiku a naše národní hospodářství vyřešiti slušně

({'komora': 'ns',
  'obdobi': 1929,
  'schuze': 247,
  'cast': 2,
  'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
  'datum': '1933-02-10',
  'komora_komplet': 'NS RČS 1929-1935',
  'schuze_komplet': '247. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
   'mluvci_id': None,
   'text': 'Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, co jsem již řekl, že třeba velmi litovati, že tento úvěr přichází teprve nyní. Pokud jde o prvou část, o pojištění vývozních úvěrů ve vlastním slova smyslu, která se netýká zaručení úvěrů zahraničním veřejným korporacím, nýbrž soukromníkům, vlastně se zde o ni nejedná. Bylo by třeba, aby i v tomto oboru zavládla jakási velkorysost. Ovšem jest to velmi choulostivá otázka. Neboť právě tehdy, když jde o státní nebo veřejné peníze, mají lidé sklon k jakési rozhazovačnosti a velkor

In [89]:
oscrapovani_stenozaznamu("2010ps_stenprot_054schuz_s054069.htm", zdroj=True)

____
____ 
TUČNÉ
____(Jednání pokračovalo v 11.02 hodin.)
____ 
____Předsedkyně PSP Miroslava Němcová  Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci v
TUČNÉ
ODKAZ
mluvčí p.find(a)
____ 
____Budeme moci zahájit projednávání bodu číslo 111. Je to 
____ 
____111.  Vládní návrh, kterým se předkládá Parlamentu České republiky  k vyslovení souhlasu s ratifikací Protokol proti pašování přis
TUČNÉ
ODKAZ
____Prosím, aby se slova ujal ministr vnitra Jan Kubice. Prosím, pane ministře. (Slovo požaduje poslanec Filip pro faktickou poznámku.
____Pardon, pan kolega - faktická poznámka může být vznesena pouze v průběhu rozpravy, a rozprava se nekoná. (Poslanec Filip uplatňuje
____ 
____Poslanec Vojtěch Filip: Děkuji, paní předsedkyně. Já jsem rád, že tady máme pana ministra vnitra, ale nemyslím si, že je možné se 
TUČNÉ
mluvčí (: + a/b)
____ 
____Předsedkyně PSP Miroslava Němcová  Děkuji. Máme tedy dvě možnosti. Já se musím zeptat pana ministra, zda je o

({'komora': 'ps',
  'obdobi': 2010,
  'schuze': 54,
  'cast': 69,
  'soubor': '2010ps_stenprot_054schuz_s054069.htm',
  'datum': '2013-06-13',
  'komora_komplet': 'Parlament České republiky, Poslanecká sněmovna 2010 – 2013',
  'schuze_komplet': '54. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci_id': None,
   'text': '(Jednání pokračovalo v 11.02 hodin.)'},
  {'poradi': 2,
   'soubor': '2010ps_stenprot_054schuz_s054069.htm',
   'mluvci': 'Předsedkyně PSP Miroslava Němcová',
   'mluvci_id': None,
   'text': 'Vážené paní poslankyně, vážení páni poslanci, budeme pokračovat v našem dnešním jednání. Chci vás seznámit s omluvami, které mi byly ještě doručeny. Obdržela jsem omluvu pana ministra financí Miroslava Kalouska, který se omlouvá z jednání dnes odpoledne. Pak tady mám upozornění, že se omlouvá také pan ministr Blažek. Ano, pan ministr Blažek. To byly omluvy, které mezi tím byly d

In [90]:
oscrapovani_stenozaznamu("2021ps_stenprot_108schuz_s108004.htm", zdroj=True)

____
____ 
TUČNÉ
____ Mimochodem, to, co předvádí Fialova vláda a to, co se zase událo za těch několik dní nebo za ten krátký časový úsek, co tady ve S
____To všechno je samozřejmě pro hnutí SPD zcela nepřijatelné. My proto navrhujeme naopak zvýšit podporu všech občanů, kteří se zodpov
____Další záležitostí je, že teď podle aktuálních čísel se reálné mzdy za Fialovy vlády propadly téměř o 20 %. Přesto Fialova vláda pl
____My jako SPD v této souvislosti - a jsou to jasná čísla - navrhujeme, aby se v té první stupnici zvýšily platy o 15 %, což je v pod
____Podle našich kvalifikovaných propočtů by se zaměstnancům státu zařazeným v první stupnici, jak už jsem říkal, měly ty základní pla
____Jak jsem říkal, tak vy sami jste si tady vloni, Fialova vládní koalice, přidali platy politikům. My jsme hlasovali proti. Sami sob
____Ještě zmíním, že Fialova vláda opravdu likviduje všechny možné segmenty našich občanů, protože i třeba když se podíváme na zeměděl
____Tady u toho je pozoruhodné, že prá

({'komora': 'ps',
  'obdobi': 2021,
  'schuze': 108,
  'cast': 4,
  'soubor': '2021ps_stenprot_108schuz_s108004.htm',
  'datum': '2024-06-25',
  'komora_komplet': 'PČR, PS 2021-...',
  'schuze_komplet': '108. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2021ps_stenprot_108schuz_s108004.htm',
   'mluvci_id': None,
   'text': 'Mimochodem, to, co předvádí Fialova vláda a to, co se zase událo za těch několik dní nebo za ten krátký časový úsek, co tady ve Sněmovně naposled byla řádná schůze, tak teď od 1. července Fialova vláda se rozhodla opět poškodit současné i budoucí důchodce zhoršením podmínek penzijního spoření. SPD naopak prosazuje zvýšení státní podpory na důchodové spoření. Od 1. července se na návrh Fialovy vlády mění podmínky státní podpory penzijního spoření. Takže například lidem, kteří by chtěli penzijní spoření v důsledku zhoršení jeho podmínek ukončit předčasně, dokonce hrozí ztráta získaných státních příspěvků. Jiný

In [109]:
oscrapovani_stenozaznamu("1929ns_ps_stenprot_247schuz_s247002.htm", zdroj=True)

____
____ Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, c
____ Pokud jde o pojištění prostřednictvím státní záruky, přednesl bych zvláštní přání. Vývojem v jednotlivých státech, neschopností p
____ Dnešní své vývody končím přáním, aby náš těžce ohrožený vývozní průmysl získal oné podpory státu, která odpovídá správnému porozu
____ Místopředseda Špatný (zvoní): Dalším řečníkem je pan posl. Richter. Uděluji mu slovo.  
TUČNÉ
počítám tučnost
MÍSTOPŘEDSEDA
mluvčí (: + a/b)
____ Posl. Richter: Slavná sněmovno! Dvouparagrafový zákon, kterým se pro příští léta stanoví úhrn státních záruk za exportní úvěry, j
TUČNÉ
počítám tučnost
POSL.
mluvčí (: + a/b)
____ Ujal jsem se však slova z jiného důvodu, než abych rozebíral předloženou osnovu. Její přednosti i nedostatky už vytkli páni zprav
TUČNÉ
____ Považuji za jednu z nejdůležitějsích otázek pro naši hospodářskou politiku a naše národní hospodářství vyřešiti slušně

({'komora': 'ns',
  'obdobi': 1929,
  'schuze': 247,
  'cast': 2,
  'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
  'datum': '1933-02-10',
  'komora_komplet': 'NS RČS 1929-1935',
  'schuze_komplet': '247. schůze',
  'prepsano': True,
  'autorizovano': True},
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1929ns_ps_stenprot_247schuz_s247002.htm',
   'mluvci_id': None,
   'text': 'Pokud jde o zákon sám, pohybuje se ve velmi skrovných hranicích. Státní záruka 600 milionů Kč není veliká a mohu jen opakovati, co jsem již řekl, že třeba velmi litovati, že tento úvěr přichází teprve nyní. Pokud jde o prvou část, o pojištění vývozních úvěrů ve vlastním slova smyslu, která se netýká zaručení úvěrů zahraničním veřejným korporacím, nýbrž soukromníkům, vlastně se zde o ni nejedná. Bylo by třeba, aby i v tomto oboru zavládla jakási velkorysost. Ovšem jest to velmi choulostivá otázka. Neboť právě tehdy, když jde o státní nebo veřejné peníze, mají lidé sklon k jakési rozhazovačnosti a velkor

In [92]:
def process_file(soubor):
    try:
        html2data(soubor)
    except Exception as e:
        print(f"{soubor}: {e}")

In [93]:
directory = "downloads/schuze"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
# files = [f for f in files if "199" in f]
# files = [f for f in files if ('1992fs_sl' in f) or ('1990fs_sl' in f)] 
total_files = len(files)

batch_size = 8

# Process files in batches to reduce memory usage and improve progress tracking
for i in range(0, total_files, batch_size):
    batch = files[i:i + batch_size]
    
    # Use ProcessPoolExecutor for CPU-bound tasks or ThreadPoolExecutor for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map is more memory efficient than list comprehension for large batches
        futures = {executor.submit(process_file, f): f for f in batch}
        
        for j, future in enumerate(concurrent.futures.as_completed(futures), 1):
            try:
                future.result()  # Get the result to raise any exceptions
            except Exception as e:
                print(f"Error processing file {futures[future]}: {e}")
            
            # Update progress every batch_size files
            if (i + j) % batch_size == 0:
                print(f"{i + j}/{total_files}")

8/102820
16/102820
24/102820
32/102820
40/102820
48/102820
56/102820
64/102820
72/102820
80/102820
88/102820
96/102820
104/102820
112/102820
120/102820
128/102820
136/102820
144/102820
152/102820
160/102820
168/102820
176/102820
184/102820
192/102820
200/102820
208/102820
216/102820
224/102820
232/102820
240/102820
248/102820
256/102820
264/102820
272/102820
280/102820
288/102820
296/102820
304/102820
312/102820
320/102820
328/102820
336/102820
344/102820
352/102820
360/102820
368/102820
376/102820
384/102820
392/102820
400/102820
408/102820
416/102820
424/102820
432/102820
440/102820
448/102820
456/102820
464/102820
472/102820
480/102820
488/102820
496/102820
504/102820
512/102820
520/102820
528/102820
536/102820
544/102820
552/102820
560/102820
568/102820
576/102820
584/102820
592/102820
600/102820
608/102820
616/102820
624/102820
632/102820
640/102820
648/102820
656/102820
664/102820
672/102820
680/102820
688/102820
696/102820
704/102820
712/102820
720/102820
728/102820
736/102820
7